<a href="https://colab.research.google.com/github/toddwalters/pgaiml-python-coding-examples/blob/main/deep-learning/C7/Transformers_from_Scratch_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 42.8 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully u

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import (Add, Dense, Layer, Dropout, Embedding, LayerNormalization,
                                     MultiHeadAttention, TextVectorization, StringLookup)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers.schedules import LearningRateSchedule
import tensorflow_text as tf_text
_AUTO = tf.data.AUTOTUNE

In [ ]:
!wget https://www.manythings.org/anki/fra-eng.zip
!unzip fra-eng.zip

--2023-01-08 16:06:27--  https://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... failed: Connection timed out.
Retrying.

--2023-01-08 16:08:41--  (try: 2)  https://www.manythings.org/anki/fra-eng.zip
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... failed: Connection timed out.
Retrying.

--2023-01-08 16:10:53--  (try: 3)  https://www.manythings.org/anki/fra-eng.zip
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6720195 (6.4M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   6.41M  13.2MB/s    in 0.5s    

2023-01-08 16:12:05 (13.2 MB/s) - ‘fra-eng.zip’ saved [6720195/6720195]

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 


In [ ]:
class Config:
  DATA_FNAME = 'fra.txt'
  BATCH_SIZE = 256

  SOURCE_VOCAB_SIZE = 10000
  TARGET_VOCAB_SIZE = 10000

  MAX_POS_ENCODING = 256 # define the max positions in source and target

  ENCODER_NUM_LAYERS = 2 # number of layers for encoders
  DECODER_NUM_LAYERS = 2 # number of layers for encoders

  #Define the dimensions of models
  D_MODEL = 128

  #Number of units in FFNN
  DFF = 256

  #define the number of heads
  NUM_HEADS = 4
  DROP_RATE = 0.1

  #Number of epochs to train
  EPOCHS = 25

  #Define the output directory
  OUTPUT_DIR = 'output'

config = Config()

In [ ]:
# exp1 = Config()
# exp1.EPOCHS = 30

# Create Attention Layers

In [ ]:
class BaseAttention(Layer):

  def __init__(self, **kwargs):
    super().init()
    # Layer Definitions
    self.mha = MultiHeadAttention(**kwargs)
    self.layernorm = LayerNormalization()
    self.add = Add()

class GlobalSelfAttention(BaseAttention):
  def call(self, x):
    attentionOuputs = self.mha(query=x,
                               key = x,
                               value=x)
    x = self.add([x,attentionOuputs])
    x = self.layernorm(x)
    return x

class CrossAttention(BaseAttention):
  def call(self,x,context):
    (attentionOutputs,attentionScores) = self.mha(query=x,
                                                  key=context,
                                                  value=context,
                                                  return_attention_scores=True)
    x = self.add([x,attentionOutputs])
    x = self.layernorm(x)
    return x

class CausalAttention(BaseAttention): # Masked Attention
  def call(self,x):
    attentionOutputs = self.mha(query=x,
                                key=x,
                                value=x,
                                use_causal_mask=True)
    x = self.add([x,attentionOutputs])
    x = self.layernorm(x)
    return x

In [ ]:
import random
def load_data(file_name):
  with open(file_name, 'rt') as textFile:
    lines = textFile.readlines()
    pairs = [line.split("\t")[:-1] for line in lines]
    random.shuffle(pairs)
    source = [src for src,_ in pairs]
    target = [trgt for _,trgt in pairs]
  return (source,target)

def splitting_dataset(source,target):
  trainSize = int(len(source) * 0.8)
  valsize = int(len(source) * 0.1)

  (trainSource,trainTarget) = (source[:trainSize],target[:trainSize])
  (valSource,valTarget) = (source[trainSize:trainSize+valsize], target[trainSize:trainSize+valsize])
  (testSource,testTarget) = (source[trainSize+valsize :], target[trainSize+valsize :])

  return (
      (trainSource,trainTarget),
      (valSource,valTarget),
      (testSource,testTarget)
  )

In [ ]:
source,target = load_data(file_name=config.DATA_FNAME)

In [ ]:
source[:5]

['This is my last project.',
 'When I heard the news, I wanted to cry.',
 'I have to tell you something.',
 "It's too late to go now.",
 'Tom got married to a widow.']

In [ ]:
target[:5]

["C'est mon dernier projet.",
 "Quand j'ai appris la nouvelle, j'ai eu envie de pleurer.",
 'Je dois vous dire quelque chose.',
 'Il est maintenant trop tard pour se mettre en route.',
 'Tom a épousé une veuve.']

In [ ]:
def make_dataset(splits, batchSize,sourceTextProcessor, targetTextProcessor, train=False):
  (source,target) = splits
  dataset = tf.data.Dataset.from_tensor_slices((source,target))

  def prepare_batch(source,target):
    source = sourceTextProcessor(source)
    targetBuffer = targetTextProcessor(target)
    targetInput = targetBuffer[:,:-1]
    targetOutput = targetBuffer[:,1:]
    return (source,targetInput), targetOutput

  if train:
    dataset = (
        dataset.shuffle(dataset.cardinality().numpy())
        .batch(batchSize)
        .map(prepare_batch,_AUTO)
        .prefetch(_AUTO)
    )
  else:
    dataset = dataset.batch(batchSize).map(prepare_batch,_AUTO).prefetch(_AUTO)

  return dataset

In [ ]:
def tf_lower_and_split_punct(text):
    # split accented characters
    text = tf_text.normalize_utf8(text, "NFKD")
    text = tf.strings.lower(text)

    # keep space, a to z, and selected punctuations
    text = tf.strings.regex_replace(text, "[^ a-z.?!,]", "")

    # add spaces around punctuation
    text = tf.strings.regex_replace(text, "[.?!,]", r" \0 ")

    # strip whitespace and add [START] and [END] tokens
    text = tf.strings.strip(text)
    text = tf.strings.join(["[START]", text, "[END]"], separator=" ")

    # return the processed text
    return text

In [ ]:
# Load the dataset
source,target = load_data(file_name=config.DATA_FNAME)

# Split the dataset
(train,val,test) = splitting_dataset(source=source,target=target)

# Create Text processing layer

sourceTextProcessor = TextVectorization(standardize = tf_lower_and_split_punct ,
                                        max_tokens=config.SOURCE_VOCAB_SIZE)
sourceTextProcessor.adapt(train[0]) # source text data

TargetTextProcessor = TextVectorization(standardize = tf_lower_and_split_punct ,
                                        max_tokens=config.TARGET_VOCAB_SIZE)
TargetTextProcessor.adapt(train[1]) # target text data

In [ ]:
trainDs = make_dataset(
    splits=train,
    batchSize=config.BATCH_SIZE,
    train=True,
    sourceTextProcessor=sourceTextProcessor,
    targetTextProcessor=TargetTextProcessor
)

validDs = make_dataset(
    splits=val,
    batchSize=config.BATCH_SIZE,
    train=False,
    sourceTextProcessor=sourceTextProcessor,
    targetTextProcessor=TargetTextProcessor
)

testDs = make_dataset(
    splits=test,
    batchSize=config.BATCH_SIZE,
    train=False,
    sourceTextProcessor=sourceTextProcessor,
    targetTextProcessor=TargetTextProcessor
)

In [ ]:
trainDs

<PrefetchDataset element_spec=((TensorSpec(shape=(None, None), dtype=tf.int64, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None)), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>

In [ ]:
class FeedForward(Layer):
  def __init__(self,dff, dModel, dropoutRate=0.1, **kwargs ):
    super().__init__(**kwargs)

    #Sequential
    self.seq = Sequential([
        Dense(units=dff, activation='relu'),
        Dense(units=dModel),
        Dropout(rate=dropoutRate)
    ])

    self.add = Add()
    self.layernorm = LayerNormalization()

  def call(self,x):
    x = self.add([x,self.seq(x)])
    x = self.layernorm(x)
    return x

# Encoder

In [ ]:
class EncoderLayer(Layer):

  def __init__(self, dModel, numHeads, dff, dropoutRate=0.1, **kwargs):

    super.__init__(**kwargs)

    self.globalSelfAttention = GlobalSelfAttention(
        num_heads = numHeads,
        key_dim = dModel//numHeads,
        dropout = dropoutRate
    )

    self.ffn = FeedForward(dff=dff, dModel = dModel, dropoutRate = dropoutRate)

  def call(self,x):
    x = self.globalSelfAttention(x)
    x = self.ffn(x)
    return x

class Encoder(Layer):
  def __init__(self,
               numLayers,
               dModel,
               numHeads,
               sourceVocabSize,
               maximumPositionEncoding,
               dff, dropoutRate, **kwargs):
    super.__init__(**kwargs)
    self.dModel = dModel
    self.numLayers = numLayers

    self.positionalEmbedding = PositionalEmbedding(vocabSize = sourceVocabSize, dModel = dModel,
                                                   maximumPositionEncoding = maximumPositionEncoding)

    self.encoderLayers = [
        EncoderLayer(dModel = dModel, dff = dff, numHeads = numHeads, dropoutRate = dropoutRate) for _ in range(numLayers)
    ]

    self.dropout = Dropout(rate=dropoutRate)

  def call(self,x):
    x = self.positionalEmbedding(x)
    x = self.dropout(x)
    for encoderLayer in self.encoderLayers:
      x = encoderLayer(x=x)
    return x

In [ ]:
def positional_encoding(length,depth):
  depth = depth/2
  positions = np.arange(length)[:,np.newaxis]
  depths = np.arange(depth)[np.newaxis,:]/depth

  angleRates = 1/(10000**depths)
  angleRads = positions * angleRates
  posEncoding = np.concatenate([np.sin(angleRads), np.cos(angleRads)],axis=-1)
  return tf.cast(posEncoding,dtype=tf.float32)

class PositionalEmbedding(Layer):
  def __init__(self,vocabSize,dModel, maximomPositonEncoding,**kwargs):
    super.__init__(**kwargs)

    self.embedding = Embedding(
        input_dim = vocabSize, output_dim = dModel,mask_zero=True
    )

    self.posEncoding = position_encoding(length = maximumPositionEncoding,
                                         depth=dModel)
    self.dModel = dModel

  def call(self,x):
    seqLen = tf.shape(x)[-1]

    x = self.embedding(x)
    x *= tf.math.sqrt(tf.cast(self.DModel,tf.float32))
    x += self.posEncoding[tf.newaxis, :seqLen, : ]
    return x